In [43]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
from torch.cuda.amp import autocast

# Accelerator 인스턴스 생성
accelerator = Accelerator()

# 모델 로드, 일부 모델은 CPU에 오프로드
model_name = "checkpoints/final"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # 혼합 정밀도 사용
    offload_buffers=True,  # 일부 텐서를 CPU로 오프로드
    low_cpu_mem_usage=True,  # CPU 메모리 최적화
)

# 모델을 Accelerator가 처리하는 장치에 자동으로 배치
model = accelerator.prepare(model)

# gradient_checkpointing 활성화
model.gradient_checkpointing_enable()

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 테스트 입력
input_text = "'지적질 늑대' 책을 아이에게 읽어주고 싶은데, 어떤 책인지 알려주세요."

# 토크나이저로 입력을 변환
inputs = tokenizer(input_text, return_tensors="pt")

# GPU로 입력 데이터 이동 (Accelerator를 사용하여 자동으로 이동)
inputs = {key: value.to(accelerator.device) for key, value in inputs.items()}

# token_type_ids를 제거
inputs.pop("token_type_ids", None)

# 혼합 정밀도 활성화 (메모리 최적화)
with autocast():
    with torch.no_grad():  # 역전파 방지, 메모리 사용 최소화
        # 텍스트 생성 (max_new_tokens를 적게 설정하여 메모리 절약)
        output_ids = model.generate(
            **inputs,
            max_new_tokens=10,  # 텍스트 길이를 줄여서 메모리 사용 최적화
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

# 생성된 텍스트 디코딩
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print("Model Output:", output_text)

# 메모리 해제
torch.cuda.empty_cache()  # GPU 메모리 해제


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 236.00 MiB. GPU 

In [46]:
!nvidia-smi

Tue May  6 12:13:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.107.02             Driver Version: 550.107.02     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:A1:00.0 Off |                  N/A |
| 30%   31C    P8             24W /  350W |   24191MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
